---
title: "matplotlib demo"
format:
  html:
    code-fold: false
jupyter: python3
---

# Defining Class LinearRegression 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class LinearRegression:
    def __init__(self, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_predicted = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Data Creation 

In [ ]:
class CreatingData:
    def __init__(self, n_samples=100, noise=0.1):
        self.n_samples = n_samples
        self.noise = noise

    def create_data(self, func, low=0, high=10):
        X = np.linspace(low, high, self.n_samples)
        y = func(X) + np.random.uniform(-self.noise, self.noise, self.n_samples)
        return X, y

    def plot_data(self, X, y):
        plt.scatter(X, y, color='blue')
        plt.xlabel('X')
        plt.ylabel('y')
        plt.show()

    def plot_line(self, X, y, model):
        plt.scatter(X, y, color='blue')
        plt.plot(X, model.predict(X), color='red')
        plt.xlabel('X')
        plt.ylabel('y')
        plt.show()

    def plot_line_with_error(self, X, y, model):
        plt.scatter(X, y, color='blue')
        plt.plot(X, model.predict(X), color='red')
        plt.fill_between(X, model.predict(X) - 0.1, model.predict(X) + 0.1, color='gray', alpha=0.5)
        plt.xlabel('X')
        plt.ylabel('y')
        plt.show()

    def print_to_csv(self, X, y, name):
        data = np.column_stack((X, y))
        np.savetxt(name, data, delimiter=',')

In [ ]:
class PBT_LinearRegression:
    def __init__(self, lr=0.01, n_iters=1000, n_particles=10, decay=0.99):
        self.lr = lr
        self.n_iters = n_iters
        self.n_particles = n_particles
        self.decay = decay
        self.particles = []
        self.best_particle = None

    def fit(self, X, y, batch_size):
        n_samples, n_features = X.shape
        for _ in range(self.n_particles):
            weights = np.zeros(n_features)
            bias = 0
            self.particles.append((weights, bias))
        
        for _ in range(self.n_iters):
            for particle in self.particles:
                weights, bias = particle
                idx = np.random.choice(n_samples, batch_size)
                X_batch = X[idx]
                y_batch = y[idx]
                y_predicted = np.dot(X_batch, weights) + bias
                dw = (1 / batch_size) * np.dot(X_batch.T, (y_predicted - y_batch))
                db = (1 / batch_size) * np.sum(y_predicted - y_batch)
                weights -= self.lr * dw
                bias -= self.lr * db
                particle = (weights, bias)
            self.particles = self.pbt(self.particles)

    def pbt(self, particles):
        errors = []
        for particle in particles:
            weights, bias = particle
            y_predicted = np.dot(X, weights) + bias
            error = np.mean((y_predicted - y) ** 2)
            errors.append(error)
        best_particle = particles[np.argmin(errors)]
        worst_particle = particles[np.argmax(errors)]
        worst_weights, worst_bias = worst_particle
        for i in range(len(particles)):
            if i == np.argmin(errors):
                continue
            weights, bias = particles[i]
            weights = best_weights + np.random.uniform(-1, 1) * (best_weights - weights)
            bias = best_bias + np.random.uniform(-1, 1) * (best_bias - bias)
            particles[i] = (weights, bias)
        return particles

    def predict(self, X):
        return np.dot(X, self.best_particle[0]) + self.best_particle[1]

In [ ]:
def func(x):
    return 2 * x + 1

if __name__ == '__main__':
    data = CreatingData()
    X, y = data.create_data(func)
    data.plot_data(X, y)
    data.print_to_csv(X, y, 'data.csv')